# Tarea 20% de SPSI 2025/2026

**Asignatura:**         Seguridad y Protección de Sistemas Informáticos (SPSI)  
**Alumno:**             Martín Hernández Ruiz  
**Grupo de trabajo:**   9  
**Curso:**              2025/2026  

## Índice

1. [Introducción teórica al cifrado de Vigenère](#intro-vigenere)  
2. [Implementación en Python del criptosistema de Vigenère](#implementacion)  
   1. [Normalización del texto y representación interna](#normalizacion)  
   2. [Funciones de cifrado y descifrado](#funciones-vigenere)  
   3. [Pruebas básicas](#pruebas-basicas)  
3. [Laboratorio de criptoanálisis del Vigenère](#laboratorio)  
   1. [Índice de coincidencia e índice de Friedman](#ic-friedman)  
   2. [Análisis de Kasiski](#kasiski)  
   3. [Ruptura de las columnas como cifrados de César](#ruptura-cesar)  
4. [Aplicación al criptograma propuesto en las tareas de SPSI](#aplicacion-criptograma)  
5. [Conclusiones](#conclusiones)  

---


## 1. Introducción teórica al cifrado de Vigenère <a id="intro-vigenere"></a>

El **cifrado de Vigenère** es un ejemplo clásico de **cifrado por sustitución polialfabética**.  
En lugar de usar un único alfabeto de sustitución (como en el cifrado de César), utiliza varios alfabetos distintos,   
controlados por una **clave** repetida periódicamente. Esto tiene como objetivo principal **disfrazar las frecuencias de las letras del texto llano** y dificultar el análisis de frecuencia directo.   

Históricamente, el cifrado fue atribuido a Blaise de Vigenère, aunque la idea aparece ya en trabajos anteriores de Belasso y Trithemius. 

Durante siglos se consideró prácticamente irrompible; llegó a ser conocido como **le chiffre indéchiffrable** y a principios del siglo XX aún se afirmaba que era “imposible de romper”. Sin embargo, su reputación era inmerecida: el examen de Kasiski y, más tarde, la prueba de Friedman mostraron que es vulnerable si se analiza con las herramientas adecuadas.   

### 1.1. Definición formal básica

Sea n ∈ w* A un alfabeto de n símbolos. Una clave de Vigenère es cualquier elemento K de ex p(A)*. La función de cifrado sobre el alfabeto A de n símbolos según la clave K es E<sub>k</sub> : exp(A)* → exp(A)* definida como sigue:

  E<sub>k</sub>(s)= {(f<sup>-1</sup>)((f(s<sub>j</sub>) + f (K<sub>j</sub><sup>len(s)</sup> )) mod n)}<sub>j</sub>

donde s = ⟨s<sub>j</sub> ⟩<sub>j</sub>, K<sup>len(s)</sup> es la yuxtaposición de la expresión K consigo misma y eventual truncamiento final hasta conseguir así una expresión con la misma longitud que s. La
función de descifrado sobre ese alfabeto y clave es D<sub>K</sub> : exp(A)∗ → exp(A)∗ definida por:  

  D<sub>k</sub>(s)= {(f<sup>-1</sup>)((f(s<sub>j</sub>) - f (K<sub>j</sub><sup>len(s)</sup> )) mod n)}<sub>j</sub>

En notación compacta (como en las transparencias), la función de cifrado \(E_K\) toma una palabra \(s\) y devuelve otra palabra de la misma longitud, sumando “texto + clave” módulo \(n\), mientras que la función de descifrado \(D_K\) resta la clave. Se cumple que \(D_K \circ E_K = \mathrm{id}\), es decir, descifrar lo cifrado recupera exactamente el texto original.   

### 1.2. Idea de seguridad y debilidad principal

La idea intuitiva es que **una misma letra de texto llano no siempre se cifra como la misma letra de texto cifrado**. Por ejemplo, la letra más frecuente del inglés, `E`, puede transformarse en muchas letras distintas en posiciones diferentes del mensaje, dependiendo de la letra de la clave que se esté usando en cada caso. Esto rompe el patrón tan evidente de un cifrado monoalfabético y hace que el análisis de frecuencias simple deje de funcionar. 

Sin embargo, la gran **debilidad** del cifrado de Vigenère está precisamente en la **estructura periódica de la clave**. Si un criptoanalista consigue averiguar la **longitud de la clave** \(m\), el texto cifrado puede reorganizarse en \(m\) columnas, cada una de las cuales se comporta como un **cifrado de César independiente**. Entonces es posible aplicar técnicas de análisis de frecuencias (χ², índice de coincidencia, etc.) columna a columna y recuperar la clave completa.   

Los dos métodos clásicos que explotan esta debilidad son:

- El **examen de Kasiski**, que busca repeticiones de bloques en el criptograma y utiliza las distancias entre ellas para proponer candidatos a longitud de clave.   
- La **prueba de Friedman** (o prueba kappa), que se basa en el **índice de coincidencia** y permite estimar la longitud de la clave a partir de las frecuencias globales (o por columnas) del texto cifrado.   

En el resto del cuaderno implementaremos tanto el cifrado y descifrado de Vigenère como estas técnicas de criptoanálisis, siguiendo el enfoque desarrollado en las transparencias de teoría y los textos de Hoffstein–Pipher–Silverman y Dalkilic–Güngör.


## 2. Implementación en Python del criptosistema de Vigenère <a id="implementacion"></a>

### 2.1. Normalización del texto y representación interna <a id="normalizacion"></a>

Para cumplir el modelo matemático visto en las transparencias de SPSI , debemos garantizar que el texto llano
y la clave contienen **solo caracteres del alfabeto**. Por eso aplicamos:

- conversión a **mayúsculas**,  
- eliminación de caracteres no alfabéticos,  
- mapeo letra ↔ número.

Esto coincide con las prácticas habituales de los cifrados clásicos y con la forma en que se
presenta el algoritmo en el material de teoría.


In [47]:
import string

# Alfabeto que usaremos: letras mayúsculas A-Z
ALPHABET = string.ascii_uppercase               # LLamamos a la función de string de python que contiene todas las letras del abecedario
ALPHABET_SIZE = len(ALPHABET)


def normalize_text(text: str) -> str:           # Normaliza texto: convierte a mayúsculas y elimina caracteres que no sean A-Z
    text = text.upper()                         # Convierte todo el texto a mayúsculas
    normalized = []                             # Lista donde acumularemos solo los caracteres A-Z
    for ch in text:                             # Recorre cada carácter del texto en mayúsculas
        if ch in ALPHABET:                      # Comprueba si el carácter pertenece al alfabeto A-Z
            normalized.append(ch)               # Añade la letra válida a la lista de normalizados
    return "".join(normalized)                  # Devuelve la cadena resultante formada solo por letras A-Z


def char_to_int(ch: str) -> int:                # Convierte un carácter 'A'-'Z' a entero 0-25
    return ord(ch) - ord('A')                   # Resta el código Unicode de 'A' para mapear 'A'->0, 'B'->1, ... 'Z'->25


def int_to_char(n: int) -> str:                 # Convierte un entero 0-25 a carácter 'A'-'Z' (con módulo 26)
    n = n % ALPHABET_SIZE                       # Normaliza el entero aplicando módulo ALPHABET_SIZE para asegurar 0-25
    return chr(n + ord('A'))                    # Suma el código de 'A' y convierte el entero a carácter correspondiente


def normalize_key(key: str) -> str:             # Normaliza la clave (solo letras A-Z, mayúscula) y valida no vacía
    k = normalize_text(key)                     # Llama a normalize_text para obtener la clave en A-Z sin caracteres no permitidos
    if not k:                                    # Comprueba si la clave normalizada está vacía
        raise ValueError("La clave debe " \
        "contener " \
        "al menos una letra A-Z.")               # Lanza excepción si no hay letras válidas
    return k                                     # Devuelve la clave normalizada y validada





### 2.2. Funciones de cifrado y descifrado <a id="funciones-vigenere"></a>

Implementamos ahora las funciones principales del criptosistema de Vigenère:
- `vigenere_encrypt(plaintext, key)`
- `vigenere_decrypt(ciphertext, key)`


In [51]:
def vigenere_encrypt(text: str, key: str) -> str:       # Define la función de cifrado Vigenère
    P = normalize_text(text)                            # Normaliza el texto de entrada (solo A-Z, mayúsculas)
    K = normalize_key(key)                              # Normaliza y valida la clave (no vacía, A-Z)
    cadena_cifrar = []                                  # Lista donde acumularemos los caracteres cifrados
    key_len = len(K)                                    # Longitud de la clave normalizada

    for i, ch in enumerate(P):                          # Recorre cada carácter del texto normalizado con su índice
        letra_val = char_to_int(ch)                     # Convierte la letra del texto a su valor numérico 0-25
        key_val = char_to_int(K[i % key_len])           # Toma la letra de la clave correspondiente (repetida) y la convierte a número
        cif_val = (letra_val + key_val) % ALPHABET_SIZE # Suma texto+clave módulo tamaño del alfabeto para cifrar
        cif_char = int_to_char(cif_val)                 # Convierte el valor numérico cifrado de vuelta a letra
        cadena_cifrar.append(cif_char)                  # Añade la letra cifrada a la lista de salida

    texto_cifrado = "texto_cifrado.txt"                 # Nombre del archivo donde se guardará el texto cifrado    
    with open(texto_cifrado, 'w+') as archivo:          # Abre (o crea) el archivo de texto cifrado en modo escritura
        archivo.write("".join(cadena_cifrar))           # Escribe la cadena cifrada en el archivo

    return "".join(cadena_cifrar)                       # Une y devuelve la cadena cifrada final


def vigenere_decrypt(cadena_cif: str, key: str) -> str: # Define la función de descifrado Vigenère
    C = normalize_text(cadena_cif)                      # Normaliza el texto cifrado (solo A-Z, mayúsculas)
    K = normalize_key(key)                              # Normaliza y valida la clave usada para descifrar
    descif_chars = []                                   # Lista donde acumularemos los caracteres descifrados
    key_len = len(K)                                    # Longitud de la clave normalizada

    for i, ch in enumerate(C):                          # Recorre cada carácter del texto cifrado con su índice
        char_val = char_to_int(ch)                      # Convierte la letra cifrada a su valor numérico 0-25
        key_val = char_to_int(K[i % key_len])           # Toma la letra de la clave correspondiente y la convierte a número
        descif_val = (char_val-key_val) % ALPHABET_SIZE # Resta clave al cifrado módulo alfabeto para obtener el valor del texto llano
        descif_char = int_to_char(descif_val)           # Convierte el valor numérico descifrado de vuelta a letra
        descif_chars.append(descif_char)                # Añade la letra descifrada a la lista de salida
    return "".join(descif_chars)                        # Une y devuelve la cadena del texto llano resultante

### 2.3. Pruebas básicas <a id="pruebas-basicas"></a>

Comprobamos ahora que cifrar y luego descifrar un mensaje devuelve el texto original (normalizado).


In [ ]:
while True:
	print("|--------------|")
	print("|  1 Cifrar    |")
	print("|  2 Descifrar |")
	print("|  3 Salir     |")
	print("|--------------|")
	try:
		opt = int(input("Ingrese opcion: "))
	except ValueError:
		print("Opción inválida, introduzca un número (1-3).")
		continue

	if opt == 1:
		cadena = input('Texto a cifrar: ')
		clave = input('Clave: ')
		print(vigenere_encrypt(cadena, clave))

	elif opt == 2:
		ruta = "texto_cifrado.txt"
		try:
			with open(ruta, 'r', encoding='utf-8') as ruta_cifrada:
				contenido_cifrado = ruta_cifrada.read()
			print("CONTENIDO ARCHIVO CIFRADO:", contenido_cifrado)
		except FileNotFoundError:
			print(f"No se encontró el archivo {ruta}")
			continue

		clave = input('Clave: ')
		print(vigenere_decrypt(contenido_cifrado, clave))

	elif opt == 3:
		break

	else:
		print("Opción no válida. Elija 1, 2 o 3.")



Vemos como se ha cifrado y descifrado el texto deseado correctamente, conociendo la clave de cifrado. ¿Pero qué pasaría si no conocemos la clave de cifrado? ¿Seriamos capaces de descifrar el texto cifrado por el algorítmo de Vigenère?. Eso será lo que estudiaremos en los siguienets partados del cuaderno.

## 3. Laboratorio de criptoanálisis del Vigenère <a id="laboratorio"></a>

A continuación construiremos un pequeño laboratorio para analizar criptogramas
cifrados con Vigenère y tratar de recuperar:
- La longitud de la clave.
- La clave.
- El texto en claro.

Nos basaremos en las ideas descritas en la bibliografía clásica (Friedman, Kasiski, etc.).


### 3.1. Índice de coincidencia e índice de Friedman <a id="ic-friedman"></a>

El **índice de coincidencia (IC)** de un texto se define como:

\[
  IC = \frac{\sum_{i=0}^{25} f_i (f_i - 1)}{N (N - 1)}
\]

donde:
- \(f_i\) es la frecuencia absoluta de la i-ésima letra,
- \(N\) es el número total de letras.

Para texto en **inglés** no cifrado, el IC típico es aproximadamente \(0.066\),
mientras que para texto completamente aleatorio es de alrededor de \(0.038\).

En un Vigenère con clave de longitud \(m\), cada columna se comporta como un cifrado
de César sobre texto en claro, por lo que su IC se aproxima al de un idioma natural.

El **método de Friedman** usa el IC global del criptograma para estimar la longitud
de la clave.


In [38]:
from collections import Counter

def index_of_coincidence(text: str) -> float:
    """Calcula el índice de coincidencia de un texto (ya normalizado)."""
    T = normalize_text(text)
    N = len(T)
    if N < 2:
        return 0.0
    freqs = Counter(T)
    num = sum(f * (f - 1) for f in freqs.values())
    den = N * (N - 1)
    return num / den


def friedman_key_length_estimate(text: str, ic_lang: float = 0.066, alphabet_size: int = 26) -> float:
    """
    Estima la longitud de clave usando la fórmula de Friedman.

    ic_lang: índice de coincidencia típico del idioma (≈ 0.066 para inglés).
    """
    T = normalize_text(text)
    N = len(T)
    if N < 2:
        return 0.0

    IC = index_of_coincidence(T)
    if IC == 0:
        return 0.0

    # Fórmula clásica de Friedman para estimar m (longitud de clave)
    # m ≈ ( (ic_lang - 1/alphabet_size) * N ) / ( (N - 1)*IC - ic_lang + 1/alphabet_size )
    kappa0 = 1 / alphabet_size
    num = (ic_lang - kappa0) * N
    den = (N - 1) * IC - ic_lang + kappa0
    if den == 0:
        return 0.0
    return num / den


# Pequeña prueba de IC sobre un texto de ejemplo
texto_ejemplo = """THIS IS JUST A SMALL EXAMPLE OF ENGLISH TEXT TO SEE THE INDEX OF COINCIDENCE"""
print("Texto ejemplo normalizado:", normalize_text(texto_ejemplo))
print("IC del ejemplo:", index_of_coincidence(texto_ejemplo))
print("Estimación de longitud de clave (debería ser ~1 para texto no cifrado):",
      friedman_key_length_estimate(texto_ejemplo))


Texto ejemplo normalizado: THISISJUSTASMALLEXAMPLEOFENGLISHTEXTTOSEETHEINDEXOFCOINCIDENCE
IC del ejemplo: 0.06504494976203067
Estimación de longitud de clave (debería ser ~1 para texto no cifrado): 0.433323970874777


### 3.2. Análisis de Kasiski <a id="kasiski"></a>

El **método de Kasiski** se basa en:

1. Buscar **repeticiones** de secuencias de longitud 3 o más en el criptograma.
2. Calcular las distancias (en número de caracteres) entre apariciones consecutivas.
3. Calcular el **MCD** (máximo común divisor) y factores de esas distancias.
4. Los divisores más frecuentes son buenos candidatos a longitud de clave.

Implementamos funciones para:

- Encontrar repeticiones y distancias.
- Listar factores de las distancias.


In [39]:
from math import gcd
from collections import defaultdict

def kasiski_distances(text: str, min_len: int = 3):
    """
    Devuelve un diccionario:
        secuencia -> lista de distancias entre apariciones consecutivas.

    text debe venir ya normalizado.
    """
    T = normalize_text(text)
    positions = defaultdict(list)

    # Registramos todas las posiciones de cada subcadena de longitud >= min_len
    for n in range(min_len, 6):  # probamos longitudes 3, 4, 5...
        for i in range(len(T) - n + 1):
            seq = T[i:i+n]
            positions[seq].append(i)

    distances = {}
    for seq, pos_list in positions.items():
        if len(pos_list) >= 2:
            ds = []
            for i in range(1, len(pos_list)):
                ds.append(pos_list[i] - pos_list[i-1])
            if ds:
                distances[seq] = ds
    return distances


def factorize(n: int):
    """Devuelve la lista de factores de n (excluyendo 1 si se desea)."""
    factors = set()
    for k in range(2, n+1):
        if n % k == 0:
            factors.add(k)
    return sorted(factors)


def kasiski_factor_stats(text: str, min_len: int = 3):
    """
    Aplica Kasiski y devuelve:
        - distances: distancias por secuencia repetida
        - factor_counts: contador de factores aparecidos
    """
    distances = kasiski_distances(text, min_len=min_len)
    factor_counts = Counter()
    for seq, ds in distances.items():
        for d in ds:
            for f in factorize(d):
                factor_counts[f] += 1

    return distances, factor_counts


# Ejemplo (ilustrativo) en un texto pequeño (no real)
ejemplo_cifrado = vigenere_encrypt("ESTEESUNTEXTODEPRUEBAESTEESUNTEXTODEPRUEBA", "CLAVE")
print("Criptograma ejemplo Kasiski:", ejemplo_cifrado)
distances, factor_counts = kasiski_factor_stats(ejemplo_cifrado, min_len=3)
print("Factores más frecuentes en el ejemplo:", factor_counts.most_common(10))


Criptograma ejemplo Kasiski: GDTZIUFNOIZEOYIRCUZFCPSOIGDUIXGITJHGARPIDL
Factores más frecuentes en el ejemplo: []


### 3.3. Ruptura de las columnas como cifrados de César <a id="ruptura-cesar"></a>

Una vez tenemos un candidato para la longitud de la clave \(m\):

1. Dividimos el criptograma en \(m\) **columnas**, donde la columna \(j\)
   contiene los caracteres \(C_i\) tales que \(i \equiv j \pmod m\).
2. Cada columna se comporta como un **cifrado de César** aplicado sobre texto en claro.
3. Para cada desplazamiento posible (0–25) calculamos una medida de ajuste con las
   frecuencias típicas del idioma (por ejemplo, un \(\chi^2\)).
4. Elegimos el desplazamiento que mejor se ajusta a las frecuencias esperadas.
5. Traducimos ese desplazamiento a una letra de clave.

Implementamos ahora estas herramientas.


In [40]:
# Frecuencias aproximadas de letras en inglés (proporciones)
ENGLISH_FREQ = {
    'A': 0.08167, 'B': 0.01492, 'C': 0.02782, 'D': 0.04253, 'E': 0.12702,
    'F': 0.02228, 'G': 0.02015, 'H': 0.06094, 'I': 0.06966, 'J': 0.00153,
    'K': 0.00772, 'L': 0.04025, 'M': 0.02406, 'N': 0.06749, 'O': 0.07507,
    'P': 0.01929, 'Q': 0.00095, 'R': 0.05987, 'S': 0.06327, 'T': 0.09056,
    'U': 0.02758, 'V': 0.00978, 'W': 0.02360, 'X': 0.00150, 'Y': 0.01974,
    'Z': 0.00074,
}

def split_into_columns(text: str, key_len: int):
    """
    Divide el texto normalizado en 'key_len' columnas según su posición módulo key_len.
    """
    T = normalize_text(text)
    cols = ["" for _ in range(key_len)]
    for i, ch in enumerate(T):
        cols[i % key_len] += ch
    return cols


def chi_squared_for_shift(column: str, shift: int, freq_model=ENGLISH_FREQ) -> float:
    """
    Calcula el estadístico chi-cuadrado para un desplazamiento de César en una columna.
    Suponemos que al descifrar con ese desplazamiento obtenemos texto en claro en inglés.
    """
    N = len(column)
    if N == 0:
        return float('inf')

    # Desciframos la columna con ese shift (como si fuera César)
    decrypted = []
    for ch in column:
        c_val = char_to_int(ch)
        p_val = (c_val - shift) % ALPHABET_SIZE
        decrypted.append(int_to_char(p_val))
    decrypted_text = "".join(decrypted)

    # Conteos observados
    freqs = Counter(decrypted_text)

    chi2 = 0.0
    for letter, p_expected in freq_model.items():
        E = p_expected * N
        O = freqs.get(letter, 0)
        chi2 += (O - E) ** 2 / (E if E > 0 else 1e-9)
    return chi2


def best_shift_for_column(column: str, freq_model=ENGLISH_FREQ):
    """
    Devuelve el desplazamiento (0-25) que minimiza el chi-cuadrado para una columna.
    """
    best_shift = None
    best_chi2 = float('inf')
    for shift in range(ALPHABET_SIZE):
        chi2 = chi_squared_for_shift(column, shift, freq_model)
        if chi2 < best_chi2:
            best_chi2 = chi2
            best_shift = shift
    return best_shift, best_chi2


def guess_key_for_length(ciphertext: str, key_len: int):
    """
    A partir de una longitud de clave conocida, intenta adivinar la clave completa
    usando el análisis por columnas + chi-cuadrado.
    """
    cols = split_into_columns(ciphertext, key_len)
    key_chars = []
    for col in cols:
        shift, chi2 = best_shift_for_column(col)
        # El desplazamiento de César que mejor explica la columna se interpreta como
        # la letra de la clave correspondiente a esa columna.
        key_chars.append(int_to_char(shift))
    return "".join(key_chars)


## 4. Aplicación al criptograma propuesto en las tareas de SPSI <a id="aplicacion-criptograma"></a>

En el enunciado de las tareas de SPSI se proporciona el siguiente criptograma
cifrado mediante un criptosistema de Vigenère. A partir de lo implementado
anteriormente, debemos averiguar:

- La **longitud de la clave**.
- La **clave**.
- El **mensaje en claro**.

En primer lugar, definimos el texto cifrado en una variable de Python y lo normalizamos.


In [41]:
ciphertext_raw = """UECWKDVLOTTVACKTPVGEZQMDAMRNPDDUXLBUICAMRHOECBHSPQLVIWO
FFEAILPNTESMLDRUURIFAEQTTPXADWIAWLACCRPBHSRZIVQWOFROGTT
NNXEVIVIBPDTTGAHVIACLAYKGJIEQHGECMESNNOCTHSGGNVWTQHKBPR
HMVUOYWLIAFIRIGDBOEBQLIGWARQHNLOISQKEPEIDVXXNETPAXNZGDX
WWEYQCTIGONNGJVHSQGEATHSYGSDVVOAQCXLHSPQMDMETRTMDUXTEQQ
JMFAEEAAIMEZREGIMUECICBXRVQRSMENNWTXTNSRNBPZHMRVRDYNECG
SPMEAVTENXKEQKCTTHSPCMQQHSQGTXMFPBGLWQZRBOEIZHQHGRTOBSG
TATTZRNFOSMLEDWESIWDRNAPBFOFHEGIXLFVOGUZLNUSRCRAZGZRTTA
YFEHKHMCQNTZLENPUCKBAYCICUBNRPCXIWEYCSIMFPRUTPLXSYCBGCC
UYCQJMWIEKGTUBRHVATTLEKVACBXQHGPDZEANNTJZTDRNSDTFEVPDXK
TMVNAIQMUQNOHKKOAQMTBKOFSUTUXPRTMXBXNPCLRCEAEOIAWGGVVUS
GIOEWLIQFOZKSPVMEBLOHLXDVCYSMGOPJEFCXMRUIGDXNCCRPMLCEWT
PZMOQQSAWLPHPTDAWEYJOGQSOAVERCTNQQEAVTUGKLJAXMRTGTIEAFW
PTZYIPKESMEAFCGJILSBPLDABNFVRJUXNGQSWIUIGWAAMLDRNNPDXGN
PTTGLUHUOBMXSPQNDKBDBTEECLECGRDPTYBVRDATQHKQJMKEFROCLXN
FKNSCWANNAHXTRGKCJTTRRUEMQZEAEIPAWEYPAJBBLHUEHMVUNFRPVM
EDWEKMHRREOGZBDBROGCGANIUYIBNZQVXTGORUUCUTNBOEIZHEFWNBI
GOZGTGWXNRHERBHPHGSIWXNPQMJVBCNEIDVVOAGLPONAPWYPXKEFKOC
MQTRTIDZBNQKCPLTTNOBXMGLNRRDNNNQKDPLTLNSUTAXMNPTXMGEZKA
EIKAGQ"""
ciphertext = normalize_text(ciphertext_raw)
len(ciphertext), ciphertext[:100]


(1051,
 'UECWKDVLOTTVACKTPVGEZQMDAMRNPDDUXLBUICAMRHOECBHSPQLVIWOFFEAILPNTESMLDRUURIFAEQTTPXADWIAWLACCRPBHSRZI')

### 4.1. IC global y estimación de longitud de clave (Friedman)

Calculamos ahora el índice de coincidencia del criptograma y usamos la
fórmula de Friedman para obtener una primera aproximación de la longitud de la clave.


In [42]:
IC_cipher = index_of_coincidence(ciphertext)
friedman_estimate = friedman_key_length_estimate(ciphertext)

print("Longitud del criptograma:", len(ciphertext))
print("Índice de coincidencia:", IC_cipher)
print("Estimación de longitud de clave (Friedman):", friedman_estimate)


Longitud del criptograma: 1051
Índice de coincidencia: 0.04181233292555842
Estimación de longitud de clave (Friedman): 0.6596615813847032


### 4.2. Análisis de Kasiski sobre el criptograma

A continuación aplicamos el análisis de Kasiski para apoyar la estimación de la
longitud de la clave. Observamos los factores que aparecen con mayor frecuencia.


In [43]:
distances, factor_counts = kasiski_factor_stats(ciphertext, min_len=3)

print("Algunas secuencias repetidas y sus distancias:")
for seq, ds in list(distances.items())[:10]:
    print(seq, "->", ds)

print("\nFactores más frecuentes (candidatos a longitud de clave):")
for factor, count in factor_counts.most_common(15):
    print(factor, "->", count)


Algunas secuencias repetidas y sus distancias:
UEC -> [292]
VAC -> [507]
GEZ -> [1022]
QMD -> [238]
AMR -> [14]
NPD -> [737]
DUX -> [238]
BHS -> [49]
HSP -> [210, 91]
SPQ -> [210, 525]

Factores más frecuentes (candidatos a longitud de clave):
7 -> 103
3 -> 44
2 -> 39
14 -> 34
21 -> 34
5 -> 21
4 -> 18
35 -> 16
9 -> 15
28 -> 15
63 -> 14
15 -> 13
13 -> 12
6 -> 12
8 -> 11


### 4.3. Ruptura tentativa para longitudes de clave candidatas

Seleccionamos manualmente algunas longitudes de clave candidatas (por ejemplo, las más
frecuentes según Kasiski y cercanas a la estimación de Friedman) y probamos a adivinar
la clave completa con el análisis por columnas.


In [58]:
# Ajusta esta lista de candidatos tras inspeccionar factor_counts y friedman_estimate
candidate_lengths = [3, 4, 5, 6, 7, 8, 9, 10]

for m in candidate_lengths:
    key_guess = guess_key_for_length(ciphertext, m)
    print(f"Longitud candidata {m}: clave aproximada ->", key_guess)


Longitud candidata 3: clave aproximada -> INA
Longitud candidata 4: clave aproximada -> ILIT
Longitud candidata 5: clave aproximada -> ACPMZ
Longitud candidata 6: clave aproximada -> INAMNP
Longitud candidata 7: clave aproximada -> CAPITAN
Longitud candidata 8: clave aproximada -> CZNAAYAC
Longitud candidata 9: clave aproximada -> MNPITAAXZ
Longitud candidata 10: clave aproximada -> JCIMZAAPMP


### 4.4. Descifrado con la clave candidata

Una vez tengamos una clave candidata plausible (por ejemplo, aquella que produzca
un texto en claro con apariencia de idioma natural), realizamos el descifrado completo.
En esta celda, el alumno puede fijar manualmente la longitud de clave y la clave
que considere más razonable tras los análisis anteriores.


In [60]:
# Ejemplo: sustituye 'CLAVEAQUI' por la clave que parezca más razonable
clave_candidata = "MNPITAAXZ"  # TODO: ajustar tras el análisis

plaintext_candidate = vigenere_decrypt(ciphertext, clave_candidata)
print(plaintext_candidate[:1000])


IRNORDVOPHGGSJKTSWURKITDAPSBCOVBXLEVWPLEYHOHDPUDHXLVLXCSQWHILSOHRDESDRXVFVQSLQTWQLNOOPAWOBQPCHIHSUAWIBOVFRRHHGYFEEVLWWOAVATGDIJVLUSAYNHXVPIOGEFNSFYFVCTKTUTYNDTQKLPCCZTVURZKYTSMIRLHROZWIQLLHKNCIONLRJGDVWWEIGWLKYWAPAAONTOPDWEBRQGTYVNNJKJUDINEAWIGLRKKVVRBEPIDOSPTNRZPLYTMGVLGPIXJMIBSRLSPMECSSTTEBECLDPKCNXRSPFBAHLETNVSBOAROMRYSRLYWJGSSNSNGLLNXNFEXNLAHSSDADBZZQGWYASATNLWTAFOZWPZHTIUEEGISGWBHGKJUFOVNZROOLSIZEFALHIFOIISTTPSFVRHIMWFBSRFSOMRRYTTDZTRSCOMCTOHMWWUPUFLPNJUPCUEOFCNPPWEBDGVXXWRUWQZKDQJBGFDILNIQMWLFYTEMIRHYBHGWWRVAFCLDSYWDZHBBAEBGTDUOGQEXLVPGYYGXNUAITNIDYGOKKRBEZETROFVVHHIHYTMACLAAUSRCHBSBTSDGGYWIFRAVEWOJESZRRSPYNSOWGOLXGWQLDENOPMFTPIEYUIJELANUYPMODSJEHGMOTRGNHDWHPWEOJPQQOGTTCNGWYCTQRERLNAUGNMXNIEYTGWJSNQOWTZBJDXPKTEAIDUWTDZBPOEOOYXCRJXYBTBKDIULHKNLESDRQODQIYUPTWHZHSMVBMATDDYVRBDEUSRNDLCGUEDGJTCRDDUEUVIQMKHGFBNDENFNOGPHSUNAKYHERCJJTWSFHPEXZEDFWCLOLYPDKPOWZBEHPWIAQJWVMHEKRVEORRHPUMMVIROJDUNYABYIEONDGPAGOUVIPFLUBOHJNUPXDNBLHCMRLNWXQSVRCTOPHJTWJIFWQMMWPPYWPDVYPOTWHVNASXMCICLFKRDADEJAIDCCB

## 5. Conclusiones <a id="conclusiones"></a>

En este cuaderno se ha:

1. Implementado el **cifrado y descifrado de Vigenère** en Python.
2. Construido un pequeño **laboratorio de criptoanálisis** que incluye:
   - Cálculo del índice de coincidencia y aplicación del método de Friedman.
   - Análisis de Kasiski para estimar la longitud de la clave.
   - Ruptura de cada columna como un cifrado de César empleando frecuencias de letras.
3. Aplicado las herramientas anteriores a un **criptograma real** propuesto en las tareas
   de SPSI, permitiendo:
   - Estimar la longitud de la clave.
   - Obtener una clave candidata.
   - Aproximarse al texto en claro.

Este enfoque reproduce las ideas clásicas de la criptoanálisis manual, pero apoyadas
en herramientas computacionales que facilitan la experimentación y el análisis
sobre criptogramas más largos.
